Edwin Sanchez

# Anomaly Detection

This notebook contains code exemplifying how to do anomaly detection using an example dataset.

> I "borrowed" a lot of code from the PyTorch Metric Learning's [Examples on Google Colab](https://github.com/KevinMusgrave/pytorch-metric-learning/blob/master/examples/README.md), specifically the [MNIST using SubCenterArcFaceLoss](https://github.com/KevinMusgrave/pytorch-metric-learning/blob/master/examples/notebooks/SubCenterArcFaceMNIST.ipynb) Notebook.

In [1]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

import os
from tqdm import tqdm
import pandas as pd
import numpy as np

## Additive Angular Margin Loss

The next section contains an implementation of the loss function called "Additive Angular Margin Loss", which is a loss function designed to create feature embeddings in a space of a target size. It does this by forcing output embeddings of a certain class to be *closer* in euclidean space, all while forcing embeddings of different classes to be farther apart. It does this along the surface of a hypersphere, with dimensions equal to the number of embedding dimensions (output dimensionality). For more details, please see the Acknowledgements section of the README, which contains a reference to ArcFace, the paper that introduces this loss function.

## Define Useful Functions / Import Custom Functions

In [2]:
# Useful Functions
from load_dataset import load_dataset

In [4]:
# Load the Dataset

# the directory containing the CIC-IDS2017 dataset
DATASET_DIR : str = "./CIC-IDS2017/"

# load the dataset
test_labels, port_labels, test_features, port_only_features = load_dataset ( dataset_dir = DATASET_DIR, num_high_traffic_ports = 5 )


Loading CSV file data from the directory './CIC-IDS2017/'.


100%|██████████| 8/8 [00:02<00:00,  2.97it/s]


## Implement the Model

In [6]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func, device, train_loader, optimizer, loss_optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        loss_optimizer.zero_grad()
        embeddings = model(data)
        loss = loss_func(embeddings, labels)
        loss.backward()
        optimizer.step()
        loss_optimizer.step()
        if batch_idx % 100 == 0:
            print("Epoch {} Iteration {}: Loss = {}".format(epoch, batch_idx, loss))


### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))


device = torch.device("cuda")

batch_size = 64
print(port_only_features[0])
train_loader = torch.utils.data.DataLoader(
    port_only_features[0], batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(test_features[0], batch_size=batch_size)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 2

### pytorch-metric-learning stuff ###
loss_func = losses.SubCenterArcFaceLoss(num_classes=10, embedding_size=128).to(device)
loss_optimizer = torch.optim.Adam(loss_func.parameters(), lr=1e-4)
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)
### pytorch-metric-learning stuff ###

tensor([], size=(0, 78), dtype=torch.float16)


ValueError: num_samples should be a positive integer value, but got num_samples=0

## Train Model

In [ ]:
for epoch in range(1, num_epochs + 1):
    train(model, loss_func, device, train_loader, optimizer, loss_optimizer, epoch)
    test(port_only_features[0], test_features[0], model, accuracy_calculator)